In [2]:
import plotly.express as px
import pandas as pd 
import altair as alt 
import numpy as np
alt.data_transformers.disable_max_rows()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df = pd.read_csv('hospital_readmissions.csv')
df.shape

(25000, 17)

In [4]:
df.head(3) 

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,[70-80),8,72,1,18,2,0,0,Missing,Circulatory,Respiratory,Other,no,no,no,yes,no
1,[70-80),3,34,2,13,0,0,0,Other,Other,Other,Other,no,no,no,yes,no
2,[50-60),5,45,0,18,0,0,0,Missing,Circulatory,Circulatory,Circulatory,no,no,yes,yes,yes


In [5]:
# Cleaning 
cols = ['time_in_hospital', 'n_lab_procedures', 'n_procedures', 
        'n_medications', 'n_outpatient', 'n_inpatient', 'n_emergency']
for col in cols:
    df[col] = df[col].astype(float)

clean_df = df.dropna()

In [6]:
df.shape

(25000, 17)

In [7]:
clean_df['glucose_test'].unique()

array(['no', 'normal', 'high'], dtype=object)

In [8]:
data = {'age': clean_df['age'], 
        'glucose': clean_df['glucose_test']}
stackbar_df = pd.DataFrame(data) 
stackbar_df.head(3)

,age,glucose
0,[70-80),no
1,[70-80),no
2,[50-60),no


In [9]:
stackbar_df = clean_df.groupby(['age', 'glucose_test']).size().reset_index(name='count')
stackbar_df.head(3)

,age,glucose_test,count
0,[40-50),high,82
1,[40-50),no,2391
2,[40-50),normal,59


In [10]:
fig = px.bar(
    stackbar_df, 
    x = 'age',
    y = 'count',
    color = 'glucose_test',
    barmode = 'stack', 
    title = 'Glucose Levels By Age Group',
    height = 500,
    width = 600,
    labels = {'age': 'Age Group', 'count': 'Count', 'glucose_test': 'Glucose Levels'} 
)
#fig.show()

In [11]:
selection = alt.selection_point(fields=['glucose_test'])

bar = alt.Chart(stackbar_df).mark_bar(size=50, stroke='black', strokeWidth=0.2).encode(
    x = alt.X('age:O', title='Ages', axis=alt.Axis(labelAngle=0)),
    y = alt.Y('count:Q', title='Counts'),
    color = alt.condition(selection, 
                          alt.Color('glucose_test:N', legend = None, scale=alt.Scale(range=['red', '#d3d3d3', 'green'])), 
                          alt.value('grey')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
    tooltip = [
        alt.Tooltip('glucose_test', title='Glucose Test'),
        alt.Tooltip('count', title='Count')]
).properties(title = 'Glucose Level Amongst Age Groups',
             width = 400, 
             height = 300
            ).transform_filter(selection)

legend = alt.Chart(stackbar_df).mark_rect().encode(
    y = alt.Y('glucose_test', title='Glucose Level'),
    color = alt.condition(selection, alt.Color('glucose_test:N').legend(None), alt.value('grey'))
).add_params(
    selection
)

chart = (bar | legend).configure_title(
    font='Times New Roman',
    fontSize=15
).configure_axis(
    labelFont='Times New Roman',
    titleFont='Times New Roman'
).configure_legend(
    labelFont='Times New Roman',
    titleFont='Times New Roman'
)
chart

alt.HConcatChart(...)

In [12]:
line_df = clean_df.groupby(['time_in_hospital', 'age'])['n_lab_procedures'].mean().reset_index(name='procedure_avg_by_hospital_time').round(2)
line_df.head(3)

,time_in_hospital,age,procedure_avg_by_hospital_time
0,1.0,[40-50),33.33
1,1.0,[50-60),32.37
2,1.0,[60-70),32.24


In [13]:
input_radio = alt.binding_radio(options = [None, '[40-50)', '[50-60)', '[60-70)', '[70-80)','[80-90)', '[90-100)'],
                                    labels = ['All','[40-50]', '[50,60]', '[60,70]', '[70-80]','[80-90]', '[90-100]'],
                                     name = "Age: " )

selection1 = alt.selection_point(fields = ['age'], bind = input_radio)
selection2 = alt.selection_point(on='mouseover', fields=['age'])

line = alt.Chart(line_df).mark_line().encode(
    x = alt.X('time_in_hospital:Q', title = "Time in Hospital"),
    y = alt.Y('procedure_avg_by_hospital_time:Q', title = 'Average Number of Lab Procedures', scale=alt.Scale(domain=[20, 70])),
    color = alt.Color('age:O', title = "Age"),
    opacity = alt.condition(selection2, alt.value(1), alt.value(0.1))
).properties(title = 'Average Procedures During Hospital Stay by Age',
             width = 400, 
             height = 300
            ).add_params(selection1, selection2
                   ).transform_filter(selection1)

point = alt.Chart(line_df).mark_point(size = 8).encode(
    x = alt.X('time_in_hospital:Q', title = "Time in Hospital"),
    y = alt.Y('procedure_avg_by_hospital_time:Q', title = 'Average Number of Lab Procedures', scale=alt.Scale(domain=[20, 70])),
    color = alt.Color('age:O', title = "Age"),
    tooltip = [
        alt.Tooltip('age', title='Patient Age'), 
        alt.Tooltip('time_in_hospital', title='Time in Hospital'), 
        alt.Tooltip('procedure_avg_by_hospital_time', title= 'Average Procedure Done')],
    opacity = alt.condition(selection2, alt.value(1), alt.value(0.1))
).properties(title = 'Average Procedures During Hospital Stay by Age',
             width = 400, 
             height = 300
            ).add_params(selection1, selection2
                   ).transform_filter(selection1)

chart = (line + point).configure_title(
    font='Times New Roman',
    fontSize=15
).configure_axis(
    labelFont='Times New Roman',
    titleFont='Times New Roman'
).configure_legend(
    labelFont='Times New Roman',
    titleFont='Times New Roman'
)

chart

alt.LayerChart(...)

In [16]:
# Parallel Coordinates Plot
# Change axis labels for readability
par_cols = {
    'time_in_hospital': 'Time in Hospital',
    'n_lab_procedures': 'Lab Procedures',
    'n_procedures': 'Procedures',
    'n_medications': 'Medications'
}

# Create a copy and rename columns for readability
par_df = clean_df[list(par_cols.keys()) + ['readmitted']].rename(columns=par_cols)

# 2. Normalize numeric columns
for c in par_cols.values():
    par_df[c] = (par_df[c] - par_df[c].min()) / (par_df[c].max() - par_df[c].min())

# 3. Sample for visual clarity
par_df_sample = par_df.sample(500, random_state=0).reset_index(drop=True)

# 4. Melt to long format
par_long = par_df_sample.reset_index().melt(
    id_vars=['index', 'readmitted'],
    value_vars=list(par_cols.values()),
    var_name='Attribute',
    value_name='Value'
)

# 5. Selection for highlighting
readmit_sel = alt.selection_point(fields=['readmitted'], bind='legend')

parallel = (
    alt.Chart(par_long)
    .mark_line()
    .encode(
        x=alt.X('Attribute:N', title='Attribute'),
        y=alt.Y('Value:Q', title='Normalized Value (0–1)', scale=alt.Scale(domain=[0, 1])),
        color=alt.Color('readmitted:N', title='Readmitted'),
        detail='index:N',
        opacity=alt.condition(readmit_sel, alt.value(0.6), alt.value(0.05)),
        tooltip=[
            alt.Tooltip('readmitted:N', title='Readmitted'),
            alt.Tooltip('Attribute:N', title='Attribute'),
            alt.Tooltip('Value:Q', title='Normalized Value', format='.2f')
        ]
    )
    .properties(
        title='Parallel Coordinates: Hospital Stay, Labs, Procedures, Medications vs Readmission',
        width=650,
        height=350
    )
    .add_params(readmit_sel)
    .configure_title(font='Times New Roman', fontSize=16)
    .configure_axis(labelFont='Times New Roman', titleFont='Times New Roman')
    .configure_legend(labelFont='Times New Roman', titleFont='Times New Roman')
)

parallel.save('parallel_coordinates_readmission.html')

parallel

alt.Chart(...)

In [17]:
#Heatmap: Readmission Rate by Diagnosis & Specialty

# 1. Compute readmission rate (% yes)
heat_df = (
    clean_df.groupby(['diag_1', 'medical_specialty'])['readmitted']
    .apply(lambda x: (x == 'yes').mean())
    .reset_index(name='readmission_rate')
)

# 2. Build heatmap
heatmap = (
    alt.Chart(heat_df)
    .mark_rect()
    .encode(
        x=alt.X(
            'diag_1:N',
            title='Primary Diagnosis Code',
            axis=alt.Axis(labelAngle=0)
        ),
        y=alt.Y('medical_specialty:N', title='Medical Specialty'),
        color=alt.Color(
            'readmission_rate:Q',
            title='Readmission Rate',
            scale=alt.Scale(scheme='greens')
        ),
        tooltip=[
            alt.Tooltip('diag_1', title='Diagnosis'),
            alt.Tooltip('medical_specialty', title='Specialty'),
            alt.Tooltip('readmission_rate:Q', title='Readmission Rate', format='.2f')
        ]
    )
    .properties(
        title='Readmission Rate by Diagnosis × Medical Specialty',
        width=650,
        height=450
    )
    .configure_title(
        font='Times New Roman',
        fontSize=16
    )
    .configure_axis(
        labelFont='Times New Roman',
        titleFont='Times New Roman'
    )
    .configure_legend(
        labelFont='Times New Roman',
        titleFont='Times New Roman'
    )
)


heatmap.save('heatmap_readmission_rate.html')

heatmap


alt.Chart(...)

In [18]:
import pandas as pd

# Read data
df = pd.read_csv("hospital_readmissions.csv")

# Convert to numeric
cols = ['time_in_hospital', 'n_lab_procedures', 'n_procedures', 
        'n_medications', 'n_outpatient', 'n_inpatient', 'n_emergency']

for col in cols:
    df[col] = df[col].astype(float)

# Clean data
clean_df = df.dropna().copy()


clean_df['is_readmitted'] = clean_df['readmitted'].apply(lambda x: 1 if x == 'yes' else 0)
clean_df = clean_df[clean_df['medical_specialty'] != 'Missing'].copy()
specialty_stats = clean_df.groupby('medical_specialty').filter(lambda x: len(x) > 50)
specialty_agg = specialty_stats.groupby('medical_specialty').agg(
    readmission_rate=('is_readmitted', 'mean'),
    count=('medical_specialty', 'count')
).reset_index()


top_10_specialties = specialty_agg.nlargest(10, 'readmission_rate')
overall_rate = clean_df['is_readmitted'].mean()
top_10_specialties.to_csv('specialty_readmission.csv', index=False)


print(f"Overall readmission rate: {overall_rate:.1%}")
print("\nTop 10 specialties (without Missing):")
print(top_10_specialties)

Overall readmission rate: 45.2%

Top 10 specialties (without Missing):
        medical_specialty  readmission_rate  count
2  Family/GeneralPractice          0.495218   1882
1        Emergency/Trauma          0.493899   1885
0              Cardiology          0.449965   1409
3        InternalMedicine          0.447686   3565
4                   Other          0.414790   2664
5                 Surgery          0.412201   1213
